# Solveur CVRP : Recherche Locale + VND + Recuit Simulé + Recherche Tabou

Ce notebook implémente un algorithme métaheuristique hybride pour résoudre les Problèmes de Routage de Véhicules Capacités (CVRP) **sans fenêtres temporelles**.

## Composants de l'Algorithme :
1. **Solution Initiale** : Heuristique du Plus Proche Voisin
2. **Recherche Locale** : Opérateurs d'amélioration
3. **VND (Descente de Voisinage Variable)** : Exploration systématique du voisinage
4. **Recuit Simulé** : Accepter des solutions pires avec une probabilité décroissante
5. **Recherche Tabou** : Structure de mémoire pour éviter les cycles

## Objectif :
Trouver des solutions dans un **écart de 7%** par rapport à la solution optimale.

In [1]:
# Import required libraries
import numpy as np
import vrplib
import yaml
import random
import math
import time
import os
from collections import deque
from copy import deepcopy
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès !")

Bibliothèques importées avec succès !


In [2]:
# Load configuration from YAML file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration chargée :")
print(yaml.dump(config, default_flow_style=False))

Configuration chargée :
general:
  random_seed: 42
  time_limit_seconds: 300
  verbose: true
initial_solution:
  method: nearest_neighbor
  randomness: 0.1
local_search:
  max_iterations: 1000
  max_iterations_without_improvement: 200
quality:
  penalty_capacity_violation: 10000
  target_gap_percentage: 7.0
simulated_annealing:
  alpha: 0.95
  final_temperature: 0.1
  initial_temperature: 1000.0
  iterations_per_temperature: 100
tabu_search:
  aspiration_enabled: true
  tabu_tenure: 20
  tabu_tenure_random_range: 10
vnd:
  max_iterations_without_improvement: 50
  neighborhoods:
  - swap
  - relocate
  - two_opt
  - cross_exchange



In [3]:
# Set random seed for reproducibility
random.seed(config['general']['random_seed'])
np.random.seed(config['general']['random_seed'])

print(f"Graine aléatoire définie sur : {config['general']['random_seed']}")

Graine aléatoire définie sur : 42


## Structures de Données et Fonctions Auxiliaires

In [4]:
class CVRPSolution:
    """Represents a CVRP solution with routes and cost."""
    
    def __init__(self, routes: List[List[int]], distance_matrix: np.ndarray):
        self.routes = routes
        self.distance_matrix = distance_matrix
        self.cost = self.calculate_cost()
    
    def calculate_cost(self) -> float:
        """Calculate total cost of the solution."""
        total_cost = 0
        for route in self.routes:
            if len(route) == 0:
                continue
            # Add distance from depot (0) to first customer
            total_cost += self.distance_matrix[0, route[0]]
            # Add distances between customers
            for i in range(len(route) - 1):
                total_cost += self.distance_matrix[route[i], route[i+1]]
            # Add distance from last customer back to depot
            total_cost += self.distance_matrix[route[-1], 0]
        return total_cost
    
    def update_cost(self):
        """Recalculate cost after modification."""
        self.cost = self.calculate_cost()
    
    def is_feasible(self, demands: np.ndarray, capacity: int) -> bool:
        """Check if solution satisfies capacity constraints."""
        for route in self.routes:
            route_demand = sum(demands[customer] for customer in route)
            if route_demand > capacity:
                return False
        return True
    
    def copy(self):
        """Create a deep copy of the solution."""
        return CVRPSolution([route.copy() for route in self.routes], self.distance_matrix)

print("Classe CVRPSolution définie !")

Classe CVRPSolution définie !


In [5]:
def calculate_distance_matrix(instance: Dict) -> np.ndarray:
    """Calculate distance matrix from instance data."""
    if 'edge_weight' in instance:
        # Distance matrix already provided
        return instance['edge_weight']
    elif 'node_coord' in instance:
        # Calculate Euclidean distances
        coords = np.array(instance['node_coord'])
        n = len(coords)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist = np.sqrt(np.sum((coords[i] - coords[j])**2))
                    dist_matrix[i, j] = dist
        return dist_matrix
    else:
        raise ValueError("Instance must have either 'edge_weight' or 'node_coord'")

print("Fonction de calcul de la matrice de distance définie !")

Fonction de calcul de la matrice de distance définie !


## Construction de la Solution Initiale

In [6]:
def nearest_neighbor_solution(instance: Dict, distance_matrix: np.ndarray) -> CVRPSolution:
    """Create initial solution using nearest neighbor heuristic."""
    n_customers = instance['dimension'] - 1  # Exclude depot
    capacity = instance['capacity']
    demands = np.array(instance['demand'])
    
    unvisited = set(range(1, n_customers + 1))
    routes = []
    
    while unvisited:
        route = []
        current_load = 0
        current_node = 0  # Start from depot
        
        while True:
            # Find nearest feasible customer
            best_customer = None
            best_distance = float('inf')
            
            for customer in unvisited:
                if current_load + demands[customer] <= capacity:
                    dist = distance_matrix[current_node, customer]
                    # Add randomness based on config
                    randomness = config['initial_solution']['randomness']
                    dist *= (1 + randomness * random.random())
                    
                    if dist < best_distance:
                        best_distance = dist
                        best_customer = customer
            
            if best_customer is None:
                break  # No feasible customer, close route
            
            route.append(best_customer)
            current_load += demands[best_customer]
            current_node = best_customer
            unvisited.remove(best_customer)
        
        if route:
            routes.append(route)
    
    return CVRPSolution(routes, distance_matrix)

print("Fonction de solution du plus proche voisin définie !")

Fonction de solution du plus proche voisin définie !


## Opérateurs de Voisinage

In [7]:
def swap_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Swap two customers between different routes."""
    if len(solution.routes) < 2:
        return None
    
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            
            if len(route_i) == 0 or len(route_j) == 0:
                continue
            
            for pos_i in range(len(route_i)):
                for pos_j in range(len(route_j)):
                    # Create new solution
                    new_solution = solution.copy()
                    
                    # Swap customers
                    new_solution.routes[i][pos_i], new_solution.routes[j][pos_j] = \
                        new_solution.routes[j][pos_j], new_solution.routes[i][pos_i]
                    
                    # Check feasibility
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Opérateur d'échange défini !")

Opérateur d'échange défini !


In [8]:
def relocate_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Relocate a customer from one route to another."""
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        route_i = solution.routes[i]
        if len(route_i) == 0:
            continue
            
        for pos_i in range(len(route_i)):
            customer = route_i[pos_i]
            
            for j in range(len(solution.routes)):
                if i == j:
                    continue
                
                for pos_j in range(len(solution.routes[j]) + 1):
                    # Create new solution
                    new_solution = solution.copy()
                    
                    # Remove customer from route i
                    removed = new_solution.routes[i].pop(pos_i)
                    
                    # Insert into route j
                    new_solution.routes[j].insert(pos_j, removed)
                    
                    # Check feasibility
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Opérateur de relocalisation défini !")

Opérateur de relocalisation défini !


In [9]:
def two_opt_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Apply 2-opt improvement within each route."""
    best_solution = None
    best_cost = solution.cost
    
    for route_idx in range(len(solution.routes)):
        route = solution.routes[route_idx]
        if len(route) < 2:
            continue
        
        for i in range(len(route) - 1):
            for j in range(i + 1, len(route)):
                # Create new solution
                new_solution = solution.copy()
                
                # Reverse segment between i and j
                new_solution.routes[route_idx][i:j+1] = reversed(new_solution.routes[route_idx][i:j+1])
                
                # No need to check feasibility (2-opt preserves route assignment)
                new_solution.update_cost()
                if new_solution.cost < best_cost:
                    best_cost = new_solution.cost
                    best_solution = new_solution
    
    return best_solution

print("Opérateur 2-opt défini !")

Opérateur 2-opt défini !


In [10]:
def cross_exchange_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Exchange segments between two routes."""
    if len(solution.routes) < 2:
        return None
    
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            
            if len(route_i) < 2 or len(route_j) < 2:
                continue
            
            # Try exchanging segments of length 1 or 2
            for seg_len in [1, 2]:
                for pos_i in range(len(route_i) - seg_len + 1):
                    for pos_j in range(len(route_j) - seg_len + 1):
                        # Create new solution
                        new_solution = solution.copy()
                        
                        # Extract segments
                        seg_i = new_solution.routes[i][pos_i:pos_i+seg_len]
                        seg_j = new_solution.routes[j][pos_j:pos_j+seg_len]
                        
                        # Exchange segments
                        new_solution.routes[i][pos_i:pos_i+seg_len] = seg_j
                        new_solution.routes[j][pos_j:pos_j+seg_len] = seg_i
                        
                        # Check feasibility
                        if new_solution.is_feasible(demands, capacity):
                            new_solution.update_cost()
                            if new_solution.cost < best_cost:
                                best_cost = new_solution.cost
                                best_solution = new_solution
    
    return best_solution

print("Opérateur d'échange croisé défini !")

Opérateur d'échange croisé défini !


## Descente de Voisinage Variable (VND)

In [11]:
def vnd(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> CVRPSolution:
    """Variable Neighborhood Descent algorithm."""
    neighborhoods = {
        'swap': swap_operator,
        'relocate': relocate_operator,
        'two_opt': two_opt_operator,
        'cross_exchange': cross_exchange_operator
    }
    
    neighborhood_order = config['vnd']['neighborhoods']
    max_no_improve = config['vnd']['max_iterations_without_improvement']
    
    current_solution = solution
    k = 0  # Neighborhood index
    no_improve_count = 0
    
    while k < len(neighborhood_order) and no_improve_count < max_no_improve:
        neighborhood_name = neighborhood_order[k]
        operator = neighborhoods[neighborhood_name]
        
        # Apply operator
        new_solution = operator(current_solution, demands, capacity)
        
        if new_solution is not None and new_solution.cost < current_solution.cost:
            current_solution = new_solution
            k = 0  # Reset to first neighborhood
            no_improve_count = 0
        else:
            k += 1  # Try next neighborhood
            no_improve_count += 1
    
    return current_solution

print("Fonction VND définie !")

Fonction VND définie !


## Recherche Tabou

In [12]:
class TabuList:
    """Tabu list for tracking forbidden moves."""
    
    def __init__(self, tenure: int):
        self.tenure = tenure
        self.tabu_dict = {}  # (move_description) -> iteration_when_expires
        self.current_iteration = 0
    
    def add(self, move: Tuple, tenure_variation: int = 0):
        """Add a move to the tabu list."""
        actual_tenure = self.tenure + random.randint(-tenure_variation, tenure_variation)
        self.tabu_dict[move] = self.current_iteration + actual_tenure
    
    def is_tabu(self, move: Tuple) -> bool:
        """Check if a move is tabu."""
        if move not in self.tabu_dict:
            return False
        return self.tabu_dict[move] > self.current_iteration
    
    def increment_iteration(self):
        """Move to next iteration and clean expired moves."""
        self.current_iteration += 1
        # Remove expired moves
        expired = [move for move, expiration in self.tabu_dict.items() 
                   if expiration <= self.current_iteration]
        for move in expired:
            del self.tabu_dict[move]

print("Classe TabuList définie !")

Classe TabuList définie !


## Recuit Simulé avec Recherche Tabou

In [13]:
def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    """Calculate acceptance probability for SA."""
    if new_cost < current_cost:
        return 1.0
    return math.exp((current_cost - new_cost) / temperature)

def simulated_annealing_with_tabu(
    initial_solution: CVRPSolution, 
    demands: np.ndarray, 
    capacity: int,
    time_limit: float = None
) -> Tuple[CVRPSolution, List[float]]:
    """Hybrid Simulated Annealing with Tabu Search and VND."""
    
    # Parameters from config
    temp = config['simulated_annealing']['initial_temperature']
    final_temp = config['simulated_annealing']['final_temperature']
    alpha = config['simulated_annealing']['alpha']
    iterations_per_temp = config['simulated_annealing']['iterations_per_temperature']
    
    tabu_tenure = config['tabu_search']['tabu_tenure']
    tabu_tenure_variation = config['tabu_search']['tabu_tenure_random_range']
    aspiration_enabled = config['tabu_search']['aspiration_enabled']
    
    max_iterations = config['local_search']['max_iterations']
    max_no_improve = config['local_search']['max_iterations_without_improvement']
    
    # Initialize
    current_solution = initial_solution
    best_solution = current_solution.copy()
    tabu_list = TabuList(tabu_tenure)
    
    cost_history = [best_solution.cost]
    no_improve_count = 0
    total_iterations = 0
    
    start_time = time.time()
    
    # Neighborhood operators
    operators = [swap_operator, relocate_operator, two_opt_operator, cross_exchange_operator]
    
    while temp > final_temp and total_iterations < max_iterations:
        # Check time limit
        if time_limit and (time.time() - start_time) > time_limit:
            break
        
        if no_improve_count >= max_no_improve:
            break
        
        for _ in range(iterations_per_temp):
            # Apply VND periodically
            if total_iterations % 50 == 0:
                current_solution = vnd(current_solution, demands, capacity)
            
            # Select random operator
            operator = random.choice(operators)
            new_solution = operator(current_solution, demands, capacity)
            
            if new_solution is None:
                continue
            
            # Create move identifier (simplified)
            move_id = (operator.__name__, hash(str(new_solution.routes)))
            
            # Check tabu status
            is_tabu = tabu_list.is_tabu(move_id)
            
            # Aspiration criterion: accept tabu move if it's better than best
            aspiration = aspiration_enabled and new_solution.cost < best_solution.cost
            
            # Decide acceptance
            if (not is_tabu or aspiration):
                # SA acceptance criterion
                if random.random() < acceptance_probability(current_solution.cost, new_solution.cost, temp):
                    current_solution = new_solution
                    tabu_list.add(move_id, tabu_tenure_variation)
                    
                    # Update best solution
                    if current_solution.cost < best_solution.cost:
                        best_solution = current_solution.copy()
                        cost_history.append(best_solution.cost)
                        no_improve_count = 0
                        if config['general']['verbose']:
                            print(f"  Nouveau meilleur : {best_solution.cost:.2f} à l'itération {total_iterations}")
                    else:
                        no_improve_count += 1
            
            tabu_list.increment_iteration()
            total_iterations += 1
        
        # Cool down
        temp *= alpha
    
    return best_solution, cost_history

print("Fonction Recuit Simulé avec Recherche Tabou définie !")

Fonction Recuit Simulé avec Recherche Tabou définie !


## Fonction Solveur Principale

In [14]:
def solve_cvrp(instance_path: str) -> Dict:
    """Solve a CVRP instance and return results."""
    
    print(f"\n{'='*60}")
    print(f"Résolution : {os.path.basename(instance_path)}")
    print(f"{'='*60}")
    
    # Read instance
    try:
        instance = vrplib.read_instance(instance_path)
    except:
        # Try Solomon format
        instance = vrplib.read_instance(instance_path, instance_format='solomon')
    
    # Calculate distance matrix
    distance_matrix = calculate_distance_matrix(instance)
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    
    print(f"Dimension : {instance['dimension']} nœuds")
    print(f"Capacité : {capacity}")
    
    # Create initial solution
    print("\nCréation de la solution initiale...")
    initial_solution = nearest_neighbor_solution(instance, distance_matrix)
    print(f"Coût initial : {initial_solution.cost:.2f}")
    print(f"Itinéraires initiaux : {len(initial_solution.routes)}")
    
    # Apply VND for quick improvement
    print("\nApplication de VND...")
    improved_solution = vnd(initial_solution, demands, capacity)
    print(f"Coût après VND : {improved_solution.cost:.2f}")
    
    # Apply SA with Tabu
    print("\nApplication du Recuit Simulé avec Recherche Tabou...")
    time_limit = config['general']['time_limit_seconds']
    start_time = time.time()
    
    final_solution, cost_history = simulated_annealing_with_tabu(
        improved_solution, demands, capacity, time_limit
    )
    
    elapsed_time = time.time() - start_time
    
    print(f"\nCoût final : {final_solution.cost:.2f}")
    print(f"Itinéraires finaux : {len(final_solution.routes)}")
    print(f"Temps écoulé : {elapsed_time:.2f} secondes")
    
    # Read optimal solution if available
    optimal_cost = None
    gap_percentage = None
    
    sol_path = instance_path.replace('.vrp', '.sol').replace('.txt', '.sol')
    if os.path.exists(sol_path):
        try:
            optimal_solution = vrplib.read_solution(sol_path)
            optimal_cost = optimal_solution['cost']
            gap_percentage = ((final_solution.cost - optimal_cost) / optimal_cost) * 100
            
            print(f"\nCoût optimal : {optimal_cost:.2f}")
            print(f"Écart : {gap_percentage:.2f}%")
            
            if gap_percentage <= config['quality']['target_gap_percentage']:
                print(f"✓ Solution dans l'écart cible de {config['quality']['target_gap_percentage']}%")
            else:
                print(f"✗ Solution dépasse l'écart cible de {config['quality']['target_gap_percentage']}%")
        except Exception as e:
            print(f"Impossible de lire la solution optimale : {e}")
    
    return {
        'instance_name': os.path.basename(instance_path),
        'solution': final_solution,
        'cost': final_solution.cost,
        'optimal_cost': optimal_cost,
        'gap_percentage': gap_percentage,
        'n_routes': len(final_solution.routes),
        'time_seconds': elapsed_time,
        'cost_history': cost_history
    }

print("Fonction solveur principale définie !")

Fonction solveur principale définie !


## Sauvegarder la Solution dans un Fichier

In [15]:
def save_solution(result: Dict, output_dir: str = 'solutions'):
    """Save solution to a file in VRPLIB format."""
    os.makedirs(output_dir, exist_ok=True)
    
    instance_name = result['instance_name'].replace('.vrp', '').replace('.txt', '')
    output_path = os.path.join(output_dir, f"{instance_name}_computed.sol")
    
    # Prepare routes (add 1 to convert from 0-indexed to 1-indexed, excluding depot)
    routes = result['solution'].routes
    
    # Write solution
    with open(output_path, 'w') as f:
        for idx, route in enumerate(routes, 1):
            route_str = ' '.join(map(str, route))
            f.write(f"Route #{idx}: {route_str}\n")
        f.write(f"Cost {result['cost']:.0f}\n")
    
    print(f"Solution sauvegardée dans : {output_path}")
    return output_path

print("Fonction de sauvegarde de solution définie !")

Fonction de sauvegarde de solution définie !


## Test sur des Instances d'Exemple

In [16]:
# Get list of CVRP instances (exclude Solomon instances which have time windows)
data_dir = 'data'
cvrp_instances = [
    os.path.join(data_dir, f) for f in os.listdir(data_dir) 
    if f.endswith('.vrp')
]

print(f"{len(cvrp_instances)} instances CVRP trouvées :")
for inst in cvrp_instances[:10]:  # Show first 10
    print(f"  - {os.path.basename(inst)}")
if len(cvrp_instances) > 10:
    print(f"  ... et {len(cvrp_instances) - 10} autres")

12 instances CVRP trouvées :
  - A-n32-k5.vrp
  - B-n31-k5.vrp
  - CMT6.vrp
  - E-n13-k4.vrp
  - F-n72-k4.vrp
  - Golden_1.vrp
  - Li_21.vrp
  - M-n101-k10.vrp
  - ORTEC-n242-k12.vrp
  - P-n16-k8.vrp
  ... et 2 autres


In [17]:
# Solve a small instance as a test
test_instance = 'data/E-n13-k4.vrp'
result = solve_cvrp(test_instance)

# Save solution
save_solution(result)


Résolution : E-n13-k4.vrp
Dimension : 13 nœuds
Capacité : 6000

Création de la solution initiale...
Coût initial : 352.00
Itinéraires initiaux : 4

Application de VND...
Coût après VND : 290.00

Application du Recuit Simulé avec Recherche Tabou...

Coût final : 290.00
Itinéraires finaux : 4
Temps écoulé : 52.04 secondes

Coût optimal : 247.00
Écart : 17.41%
✗ Solution dépasse l'écart cible de 7.0%
Solution sauvegardée dans : solutions\E-n13-k4_computed.sol

Coût final : 290.00
Itinéraires finaux : 4
Temps écoulé : 52.04 secondes

Coût optimal : 247.00
Écart : 17.41%
✗ Solution dépasse l'écart cible de 7.0%
Solution sauvegardée dans : solutions\E-n13-k4_computed.sol


'solutions\\E-n13-k4_computed.sol'

## Résoudre Plusieurs Instances

In [18]:
# Select a subset of instances for testing
selected_instances = [
    'data/A-n32-k5.vrp',
    'data/B-n31-k5.vrp',
    'data/CMT6.vrp',
    'data/Golden_1.vrp'
]

# Filter to only existing instances
selected_instances = [inst for inst in selected_instances if os.path.exists(inst)]

results = []

for instance_path in selected_instances:
    try:
        result = solve_cvrp(instance_path)
        save_solution(result)
        results.append(result)
    except Exception as e:
        print(f"Error solving {instance_path}: {e}")
        continue

print(f"\n\nCompleted solving {len(results)} instances!")


Résolution : A-n32-k5.vrp
Dimension : 32 nœuds
Capacité : 100

Création de la solution initiale...
Coût initial : 1138.64
Itinéraires initiaux : 5

Application de VND...
Coût après VND : 797.45

Application du Recuit Simulé avec Recherche Tabou...
Coût après VND : 797.45

Application du Recuit Simulé avec Recherche Tabou...

Coût final : 797.45
Itinéraires finaux : 5
Temps écoulé : 303.16 secondes

Coût optimal : 784.00
Écart : 1.72%
✓ Solution dans l'écart cible de 7.0%
Solution sauvegardée dans : solutions\A-n32-k5_computed.sol

Résolution : B-n31-k5.vrp
Dimension : 31 nœuds
Capacité : 100

Création de la solution initiale...
Coût initial : 901.28
Itinéraires initiaux : 5

Application de VND...

Coût final : 797.45
Itinéraires finaux : 5
Temps écoulé : 303.16 secondes

Coût optimal : 784.00
Écart : 1.72%
✓ Solution dans l'écart cible de 7.0%
Solution sauvegardée dans : solutions\A-n32-k5_computed.sol

Résolution : B-n31-k5.vrp
Dimension : 31 nœuds
Capacité : 100

Création de la solu

KeyboardInterrupt: 

## Statistiques Résumé

In [19]:
# Create summary report
import pandas as pd

summary_data = []
for result in results:
    summary_data.append({
        'Instance': result['instance_name'],
        'Coût Calculé': f"{result['cost']:.2f}",
        'Coût Optimal': f"{result['optimal_cost']:.2f}" if result['optimal_cost'] else 'N/A',
        'Écart (%)': f"{result['gap_percentage']:.2f}" if result['gap_percentage'] is not None else 'N/A',
        'Itinéraires': result['n_routes'],
        'Temps (s)': f"{result['time_seconds']:.2f}"
    })

df_summary = pd.DataFrame(summary_data)
print("\nRésumé des Résultats :")
print(df_summary.to_string(index=False))

# Calculate statistics for instances with known optimal
gaps = [r['gap_percentage'] for r in results if r['gap_percentage'] is not None]
if gaps:
    print(f"\n\nStatistiques d'Écart :")
    print(f"  Écart moyen : {np.mean(gaps):.2f}%")
    print(f"  Écart médian : {np.median(gaps):.2f}%")
    print(f"  Écart min : {np.min(gaps):.2f}%")
    print(f"  Écart max : {np.max(gaps):.2f}%")
    print(f"  Instances dans {config['quality']['target_gap_percentage']}% : {sum(g <= config['quality']['target_gap_percentage'] for g in gaps)}/{len(gaps)}")


Résumé des Résultats :
    Instance Coût Calculé Coût Optimal Écart (%)  Itinéraires Temps (s)
A-n32-k5.vrp       797.45       784.00      1.72            5    303.16
B-n31-k5.vrp       709.37       672.00      5.56            5    300.48
    CMT6.vrp       609.37       555.43      9.71            5    307.57


Statistiques d'Écart :
  Écart moyen : 5.66%
  Écart médian : 5.56%
  Écart min : 1.72%
  Écart max : 9.71%
  Instances dans 7.0% : 2/3


In [ ]:
# Save summary to CSV
summary_path = 'solutions/summary_results.csv'
df_summary.to_csv(summary_path, index=False)
print(f"Résumé sauvegardé dans : {summary_path}")

Summary saved to: solutions/summary_results.csv
